# News Scrapper with Multi-core : by Media

## Definition

In [1]:
import numpy as np
import pandas as pd
import re
import json
import os
import itertools as it
import pickle
import multiprocessing as mpr
import datetime as dt
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from pprint import pprint
from enum import Enum, unique

import feedparser
import urllib
from bs4 import BeautifulSoup as bs

import nltk
import konlpy as knlp
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Hannanum, Kkma, Twitter

---
### Create links of News-List page

In [2]:
def news_link_extractor(source, date):

    chosun_url_dict = {
    'chosun_economics':
    'http://news.chosun.com/svc/list_in/list.html?catid=1&source=1&indate={date}&pn={pg_num}',

    'chosun_politics':
    'http://news.chosun.com/svc/list_in/list.html?catid=2&source=1&indate={date}&pn={pg_num}',

    'chosun_national':
    'http://news.chosun.com/svc/list_in/list.html?catid=3&source=1&indate={date}&pn={pg_num}',

    'chosun_international':
    'http://news.chosun.com/svc/list_in/list.html?catid=4&source=1&indate={date}&pn={pg_num}',

    'chosun_cultures':
    'http://news.chosun.com/svc/list_in/list.html?catid=5&source=1&indate={date}&pn={pg_num}',

    'chosun_opinions':
    'http://news.chosun.com/svc/list_in/list.html?catid=6&source=1&indate={date}&pn={pg_num}',

    'chosun_accidents':
    'http://news.chosun.com/svc/list_in/list.html?catid=7&source=1&indate={date}&pn={pg_num}',

    'chosun_sports':
    'http://news.chosun.com/svc/list_in/list.html?catid=G1&source=1&indate={date}&pn={pg_num}',

    'chosun_entertainments':
    'http://news.chosun.com/svc/list_in/list.html?catid=G1&source=1&indate={date}&pn={pg_num}',
                     }
    
    press_dict = {'chosun': chosun_url_dict,
             'hani': None}
    
    src_dict = press_dict[source]
    url_dict = {'%s_%s' % (key, pg_num):
                src_dict[key].format(date=date, pg_num=pg_num)\
                for key, pg_num in it.product(src_dict, [1, 2])}

    # Get the Full article of the Feeds
#    pool = mpr.pool.Pool(processes=worker)
#    response = pool.map(link_parser, feed_list)
#    news_list = sum(response, [])
    return url_dict

In [3]:
aa = news_link_extractor('chosun', '20171128')
aa

{'chosun_accidents_1': 'http://news.chosun.com/svc/list_in/list.html?catid=7&source=1&indate=20171128&pn=1',
 'chosun_accidents_2': 'http://news.chosun.com/svc/list_in/list.html?catid=7&source=1&indate=20171128&pn=2',
 'chosun_cultures_1': 'http://news.chosun.com/svc/list_in/list.html?catid=5&source=1&indate=20171128&pn=1',
 'chosun_cultures_2': 'http://news.chosun.com/svc/list_in/list.html?catid=5&source=1&indate=20171128&pn=2',
 'chosun_economics_1': 'http://news.chosun.com/svc/list_in/list.html?catid=1&source=1&indate=20171128&pn=1',
 'chosun_economics_2': 'http://news.chosun.com/svc/list_in/list.html?catid=1&source=1&indate=20171128&pn=2',
 'chosun_entertainments_1': 'http://news.chosun.com/svc/list_in/list.html?catid=G1&source=1&indate=20171128&pn=1',
 'chosun_entertainments_2': 'http://news.chosun.com/svc/list_in/list.html?catid=G1&source=1&indate=20171128&pn=2',
 'chosun_international_1': 'http://news.chosun.com/svc/list_in/list.html?catid=4&source=1&indate=20171128&pn=1',
 'cho

---
### Crawl and Make-up the feeds of the news pages 

In [4]:
def link_parser(url):
    page = urllib.request.urlopen(url)
    soup = bs(page, "html5lib")
    html_list = soup.find_all('dl', attrs={'class': 'list_item'})
    link_list = [(link.a.text, link.a['href']) for link in html_list]
    return link_list

In [5]:
def get_news_datetime(url):
    # Chosun
    page = urllib.request.urlopen(url)
    soup = bs(page, "html5lib")
    pattern = re.compile(r'var TempDate=.+ : (\d+.\d+.\d+ \d+:\d+)"$', re.MULTILINE | re.DOTALL)
    script = soup.find("script", text=pattern)
    dt_tm_str = pattern.findall(script.text)[0]
    dt_tm = dt.datetime.strptime(dt_tm_str, '%Y.%m.%d %H:%M').strftime('%Y-%m-%d %H:%M:%S')
    return dt_tm

In [6]:
def get_news_title(url):
    # Chosun
    page = urllib.request.urlopen(url)
    soup = bs(page, "html5lib")
    pattern = re.compile(r'var ArtTitle =\s*(.+)";$')
    script = soup.find("script", text=pattern)
    title_str = pattern.findall(script.text)[0]
    return title_str

In [7]:
def link_crawler_single(url_dict):

    full_link_gen = ((key, link_parser(url_dict[key])) for key in url_dict)
    
    link_list = [{'press': key.split('_')[0],
                  'topic': key.split('_')[1],
                  'link': link,
                  'title': title,
                  'datetime': get_news_datetime(link)}\
                 for key, link_list in full_link_gen for title, link in link_list]

    return link_list

In [8]:
def link_crawler(key, url):

    full_link_tuple = ((key, link_parser(url)), )
    link_list = [{'press': key.split('_')[0],
                  'topic': key.split('_')[1],
                  'link': link,
                  'title': title,
                  'datetime': get_news_datetime(link)}\
                 for key, link_list in full_link_tuple for title, link in link_list]

    return link_list

In [9]:
bb = link_crawler_single(aa)
bb[:2]

[{'datetime': '2017-11-28 22:42:00',
  'link': 'http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900506.html',
  'press': 'chosun',
  'title': 'LH 단지내 상가 입찰 105억 뭉칫돈 몰렸네',
  'topic': 'economics'},
 {'datetime': '2017-11-28 22:22:00',
  'link': 'http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900500.html',
  'press': 'chosun',
  'title': '미친 분양가… 3.3㎡당 1억원 찍나',
  'topic': 'economics'}]

In [10]:
aa_items = [(key, item) for key, item in aa.items()]
bb = link_crawler(aa_items[0][0], aa_items[0][1])
bb[:2]

[{'datetime': '2017-11-28 22:42:00',
  'link': 'http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900506.html',
  'press': 'chosun',
  'title': 'LH 단지내 상가 입찰 105억 뭉칫돈 몰렸네',
  'topic': 'economics'},
 {'datetime': '2017-11-28 22:22:00',
  'link': 'http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900500.html',
  'press': 'chosun',
  'title': '미친 분양가… 3.3㎡당 1억원 찍나',
  'topic': 'economics'}]

---

---
### Scrap the News Articles

In [11]:
def article_html_reader(url, source):
    
    src_dict = {'chosun': None,
                'hani': None,
                'mk': None,
                'hk': None}

    try:
        page = urllib.request.urlopen(url)
        soup = bs(page, "html5lib")

        if source == 'chosun':
            raw_html = raw_html = soup.find_all('div', attrs={'par', 'article'})

        elif source == 'hani':
            raw_html = soup.find_all('div', attrs='text')
            
        elif source == 'mk':
            raw_html = soup.find_all('div', attrs='art_txt')

        elif source == 'hk':
            #raw_html = str(soup.find_all('div', attrs={'news_article', 'wrap_article'}))[1:-1]
            raw_html = soup.find_all('div', attrs={'wrap_article', 'news_article'})

    except urllib.error.HTTPError as e:
        print(url, str(e))
        raw_html = None
    
    # For to check the progress
    #print('%s (%s)' % (source, url))
    
    return raw_html


In [12]:
def article_string_extractor(bs_soup):
    
    if bs_soup.find('script'):
        bs_soup.script.decompose()

    if bs_soup.find('div', attrs='center_img'):
        bs_soup.find('div', attrs='center_img').decompose()

    if bs_soup.find('div'):    
        bs_soup.find('div').decompose()
        
    item_str = str(bs_soup)

    # Extraction
    pattern = re.compile(r'<[^<>]*>')
    extracted = ''.join(re.split(pattern, item_str))

    # Substitution
    pattern = re.compile(r"\n|\t|\s+$")
    substituted = re.sub(pattern, '', extracted)

    sliced = substituted

    return sliced

In [13]:
def news_scrapper(url, source):

    html_list = article_html_reader(url, source)
    news_str = ''.join([article_string_extractor(item) for item in html_list])
    
    return news_str

In [14]:
def scrap_from_link(feed_list):
    res = [dict(feed, **{'body': news_scrapper(feed['link'], feed['press'])})\
           for feed in feed_list]
    return res

In [15]:
cc = scrap_from_link(bb)
cc[:2]

[{'body': 'LH(한국토지주택공사)가 최근 공급한 단지 내 상가 입찰에 105억원의 뭉칫돈이 몰렸다.28일 상가정보연구소에 따르면, 지난 20~24일 진행된 LH 단지 내 상가의 입찰에서 37실 중 36실이 낙찰됐다. 낙찰가 총액은 105억5464만원, 평균 낙찰가율은 135.3%였다.경남 양산시 양산물금2지구 H1블록 103호는 예정가 1억3500만원보다 1억1500여만원 비싼 2억5065만5500원에 낙찰돼 이번 공급 상가 중 최고 낙찰가율(186%)을 기록했다. 예정가 6억3000만원이던 같은 상가 204호는 유찰됐다.LH 단지 내 상가는 일반 상가보다 가격이 저렴해 상대적으로 소액 투자가 가능하고, 고정 수요가 탄탄한 것이 장점으로 꼽힌다. 이 때문에 상반기 공급된 LH 단지 내 상가는 평균 낙찰가율이 200%에 육박하기도 했다. 1억원짜리를 2억원에 사들였다는 뜻이다. 그러나 최근 들어 시중금리가 오르면서 상가 등 수익형 부동산 투자 열기가 한풀 꺾일 것이라는 전망이 나온다.',
  'datetime': '2017-11-28 22:42:00',
  'link': 'http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900506.html',
  'press': 'chosun',
  'title': 'LH 단지내 상가 입찰 105억 뭉칫돈 몰렸네',
  'topic': 'economics'},
 {'body': '서울 한남동 외인아파트 부지에 들어서는 고급 아파트 \'나인원 한남\'의 분양가가 3.3㎡당 최고 1억원에 달한다는 사실이 알려지자 고분양가 논란이 다시 불거지고 있다. 일부에서는 "\'높은 분양가→주변 시세 상승→더 높은 분양가\'로 이어지는 악순환을 끊지 않으면 집값 안정은 불가능하다"는 목소리도 나온다. 정부도 최근 들어 HUG(주택도시보증공사)를 통한 사실상 분양가 통제에 나서고 있다. 하지만 "분양가 통제가 \'로또 아파트\'를 양산할 수 있으며, 다양성 차원에서도 고급 아파

---
### Dump the News

In [16]:
def news_list_scrapper(media, date, worker=1):

    url_dict = news_link_extractor('chosun', date)

    # Get the RSS Feeds
    # Get the Full article of the Feeds
    pool = mpr.pool.Pool(processes=worker)
    response = pool.starmap(link_crawler, url_dict.items())
    feed_list = response

    # Print Lengths of each Feeds
    feed_count_dict = {press: len(feed) for press, feed in zip(url_dict, feed_list)}
    feed_count = sum(feed_count_dict.values())
    print("Scrapping '%s, %s' : %s" % (media, date, feed_count))
    #pprint(feed_count_dict)

    
    # Get the Full article of the Feeds
    #print("Scrapping '%s'..." % date)
    pool = mpr.pool.Pool(processes=worker)
    response = pool.map(scrap_from_link, feed_list)
    news_list = sum(response, [])
    #print("\rScrapping '%s'...Done" % date)

    return news_list

In [17]:
def news_list_dumper(media, dirname, start_dt, end_dt, worker=1):

    date_list = [date.strftime('%Y%m%d')\
                 for date in pd.date_range(start=start_dt, end=end_dt)]

    news_list = sum([news_list_scrapper(media, date, worker=worker)\
                          for date in date_list], [])

    total_cnt = len(news_list)
    print("Media    : %s" % media)
    print("Date     : %s ~ %s" % (start_dt, end_dt))
    print("Total    : %s" % total_cnt)
    os.makedirs(dirname, exist_ok=True)
    date_str = dt.datetime.now().strftime('%Y%m%d')
    filename = '/'.join([dirname, 'newsdata'])
    print("Dumping '%s' article(s) to '%s.*' ..." % (total_cnt, filename))
    
    # Dump the result
    with open('%s.json' % filename, "w") as json_file:
        json_file.write(json.dumps(news_list))

    data = pd.DataFrame.from_dict(news_list)
    full_txt = '\n'.join(data['body'])
    with open('%s.txt' % filename, 'w') as text_file:
        text_file.write(full_txt)
    
    full_body_list = data['body'].tolist()
    with open('%s.dump' % filename, 'wb') as byte_file:
        pickle.dump(full_body_list, byte_file)

    print("Dumping '%s' article(s) to '%s.*' ...Done" % (total_cnt, filename))

    return news_list

## Save the data

In [18]:
dump_name = 'test'

In [19]:
if __name__ == '__main__':
    test = news_list_dumper('chosun', dump_name, '20171101', '20171130',  worker=8)

Scrapping 'chosun, 20171101' : 113
Scrapping 'chosun, 20171102' : 108
Scrapping 'chosun, 20171103' : 114
Scrapping 'chosun, 20171104' : 91
Scrapping 'chosun, 20171105' : 4
Scrapping 'chosun, 20171106' : 85
Scrapping 'chosun, 20171107' : 88
Scrapping 'chosun, 20171108' : 108
Scrapping 'chosun, 20171109' : 105
Scrapping 'chosun, 20171110' : 108
Scrapping 'chosun, 20171111' : 95
Scrapping 'chosun, 20171112' : 2
Scrapping 'chosun, 20171113' : 102
Scrapping 'chosun, 20171114' : 100
Scrapping 'chosun, 20171115' : 102
Scrapping 'chosun, 20171116' : 87
Scrapping 'chosun, 20171117' : 97
Scrapping 'chosun, 20171118' : 87
Scrapping 'chosun, 20171119' : 5
Scrapping 'chosun, 20171120' : 104
Scrapping 'chosun, 20171121' : 123
Scrapping 'chosun, 20171122' : 95
Scrapping 'chosun, 20171123' : 96
Scrapping 'chosun, 20171124' : 109
Scrapping 'chosun, 20171125' : 101
Scrapping 'chosun, 20171126' : 3
Scrapping 'chosun, 20171127' : 103
Scrapping 'chosun, 20171128' : 101
Scrapping 'chosun, 20171129' : 101
Sc

## Load the Data

In [20]:
dump_name = 'test'

### DataFrame from `json`

In [21]:
fname = dump_name + '/newsdata.json'
data = pd.read_json(fname)
data['datetime'] = pd.to_datetime(data['datetime'])
data[:2]

,body,datetime,link,press,title,topic
0,우리나라의 에너지 자원 해외 수입 의존도는 96%에 달한다. 특히 액화천연가스(LN...,2017-11-01 17:30:00,http://news.chosun.com/site/data/html_dir/2017...,chosun,"전 세계 13개국서 탐사·개발… 모잠비크선 ""금세기 최대 규모""",economics
1,'공정한 병역(兵役) 실현'을 최우선 과제로 삼은 병무청이 올해 초부터 병역 판정 ...,2017-11-01 17:18:00,http://news.chosun.com/site/data/html_dir/2017...,chosun,총 25개 항목의 종합병원급 건강검진… 실시간 공개 서비스 제공,economics


### `text`

In [22]:
fname = dump_name + '/newsdata.txt'
with open(fname, 'r') as f:
    full_txt = f.read()
full_txt[:100]

'우리나라의 에너지 자원 해외 수입 의존도는 96%에 달한다. 특히 액화천연가스(LNG)는 전량 수입한다. 에너지 안보를 위해 안정적인 자원 확보는 무엇보다 중요하다. 해외 자원 개'

### `byte` Object

In [23]:
fname = dump_name + '/newsdata.dump'
with open(fname, 'rb') as f:
    full_list = pickle.load(f)
    
full_list[:2]

['우리나라의 에너지 자원 해외 수입 의존도는 96%에 달한다. 특히 액화천연가스(LNG)는 전량 수입한다. 에너지 안보를 위해 안정적인 자원 확보는 무엇보다 중요하다. 해외 자원 개발 사업은 이제 선택이 아니라 필수가 됐다.올해로 창립 34주년을 맞은 한국가스공사는 과거 단순히 LNG를 도입·판매하던 것에서 벗어나 종합 에너지기업으로 성장하기 위해 에너지 자원 개발 역량을 키우는 데 집중하고 있다. 현재 전 세계 13개국에서 탐사, 개발·생산, LNG, 인프라 등 24개 사업을 진행하고 있다.가스공사가 해외에서 확보한 자원량은 지난해 기준 2억5158만t에 달한다. 한국이 8년 가까이 사용할 수 있는 규모다. 가스공사는 지난해 해외 사업에서 1조102억원의 매출을 기록했다. 올해 상반기엔 445억원의 영업이익을 기록했다.대표적인 해외 자원 개발 사업이 가스공사가 지난 2007년 지분 매입 계약을 한 모잠비크 4구역(Area 4) 가스전 개발 사업이다. 모잠비크 동쪽 해상에 있는 이 가스전은 \'금세기 최대 규모의 가스전\'으로 불린다. 매장량이 19억2000만t에 달한다. 가스공사는 이 가스전 지분 10%를 보유하고 있다. 가스공사의 보유 매장량을 국내 소비량으로 환산하면 5년 6개월을 쓸 수 있는 물량이다. 내년부터 본격적으로 개발이 시작되면 2022년부터 상업 생산이 이뤄질 예정이다.이라크 주바이르 유·가스전 개발 사업은 가스공사의 해외 자원 개발 사업 가운데 투자 회수율이 가장 높은 사업 가운데 하나다. 가스공사는 이 유전 지분의 약 23%를 확보하고 있다. 이곳에선 하루 최대 85만배럴의 원유가 생산된다. 가스공사는 2010년 자회사를 설립, 개발에 착수했다. 그동안 자회사에 3억7800만달러를 투자해 1단계로 올해 5월 약 2억달러를 회수했다.호주 북동부 퀸즈랜드주의 가스전 개발 및 액화플랜트 건설·운영사업도 벌이고 있다. 지분 15%를 확보, 2015년 9월 첫 LNG 생산을 개시했다.이 밖에도 가스공사는 우즈베키스탄 사상 최대 규모의 패키지형 에

Done.